## LTSM

## Failed: BERT embedding estranho 


In [4]:
import pandas as pd
import numpy as np

In [78]:
df = pd.read_csv('../embeddings/BERT_Embeddings_full.csv')
df.head()

,age,age_encoded,word_tokens,lemma,0,1,2,3,4,5,...,758,759,760,761,762,763,764,765,766,767
0,a2,1,"['quantos', 'empregados', 'em', 'cada', 'um', ...",quanto empregado em cada um de o atual nível s...,0.564466,0.815525,0.107565,0.319572,0.101842,0.105100,...,0.275671,0.219380,0.934174,0.909270,0.090046,0.897560,0.590714,0.291341,0.416531,0.900556
1,a2,1,"['solicito', 'cópia', 'das', 'atas', 'do', 'co...",solicito cópia de o ata de o conselho de admin...,0.520970,0.815053,0.120955,0.300525,0.079995,0.149684,...,0.282352,0.235483,0.916027,0.947861,0.102334,0.876205,0.603262,0.254797,0.394609,0.841805
2,a2,1,"['solicito', 'informar', 'a', 'norma', 'lei', ...",solicito informar o norma lei decreto portaria...,0.535018,0.845178,0.106620,0.330766,0.061033,0.094648,...,0.288288,0.232071,0.927528,0.953746,0.102037,0.927873,0.533769,0.266102,0.429789,0.895740
3,a2,1,"['solicito', 'por', 'gentileza', 'a', 'informa...",solicito por gentileza o informação sobre o qu...,0.548547,0.784378,0.057764,0.275971,0.123137,0.096042,...,0.306542,0.244469,0.954206,0.941051,0.075034,0.899374,0.585249,0.279545,0.380809,0.939743
4,a2,1,"['solicito', 'por', 'gentileza', 'a', 'informa...",solicito por gentileza o informação sobre o qu...,0.559063,0.799921,0.060454,0.274691,0.135161,0.079634,...,0.304113,0.247853,0.955085,0.938762,0.081348,0.894296,0.595016,0.278718,0.383074,0.934677


In [79]:
df.groupby('age')['age_encoded'].count()

age
a1    1200
a2    2000
a3    3000
a4    2000
Name: age_encoded, dtype: int64

In [80]:
df_label = pd.get_dummies(df['age'])
df_text = df.drop(['word_tokens', 'lemma', 'age', 'age_encoded'], axis = 1)

## Model

In [3]:
import tensorflow as tf
#from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [7]:
max_features = 700
embed_dim = 32

In [8]:
batch_size = 32
early_stopping = EarlyStopping(monitor='val_loss', patience= 3)

In [85]:
def modelLSTMPos(df_shape):
    
    model = Sequential([
        layers.Embedding(max_features, embed_dim, input_length= df_shape.shape[1]),
        #layers.Conv1D(filters=32, kernel_size=3, padding='same', activation= 'relu'),
        #layers.MaxPooling1D(pool_size=2),
        layers.LSTM(64, dropout=0.4, recurrent_dropout=0.2),
        layers.Dense(64, activation='relu'),
        layers.Dense(4, activation='softmax')
    ])

    #optimizer = Adam(learning_rate=3e-4)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [86]:
model = modelLSTMPos(df)

In [87]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_8 (Embedding)     (None, 772, 32)           22400     
                                                                 
 lstm_8 (LSTM)               (None, 64)                24832     
                                                                 
 dense_16 (Dense)            (None, 64)                4160      
                                                                 
 dense_17 (Dense)            (None, 4)                 260       
                                                                 
Total params: 51652 (201.77 KB)
Trainable params: 51652 (201.77 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### BERT - Train, Test split

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier

2023-12-17 11:52:11.219679: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-17 11:52:11.259893: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-17 11:52:11.259939: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-17 11:52:11.259974: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-17 11:52:11.269165: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-17 11:52:11.269904: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [89]:
X_train, X_test, y_train, y_test = train_test_split(df_text, df_label, train_size=0.8, random_state=42 )
X_train.shape


(6560, 768)

In [90]:
model = modelLSTMPos(df_text)

In [ ]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

### BERT - K-fold

In [33]:
from sklearn.model_selection import KFold

In [34]:
# clf = KerasClassifier(build_fn=modelLSTMPos(df_text), epochs=10, batch_size=32, verbose=1)
# resultPos = cross_val_score(clf, X_train, y_train, scoring='accuracy', cv=10)
# print(resultPos)
# resultPos.mean()

### Plain text - Train, test

In [5]:

df_plain = pd.read_csv('../data/enc_tok_nostopword_nopunct_lemm.csv')
df_plain.head()

,Unnamed: 0,req_text,age,age_encoded,word_tokens,lemma
0,0,quantos empregados em cada um dos atuais níve...,a2,1,"['quantos', 'empregados', 'cada', 'atuais', 'n...",quanto empregado cada atual nível salarial pca...
1,1,solicito cópia das atas do conselho de admini...,a2,1,"['solicito', 'cópia', 'atas', 'conselho', 'adm...",solicito cópia ata conselho administração bnde...
2,2,solicito informar a norma lei decreto portari...,a2,1,"['solicito', 'informar', 'norma', 'lei', 'decr...",solicito informar norma lei decreto portaria i...
3,3,solicito por gentileza a informação sobre a q...,a2,1,"['solicito', 'gentileza', 'informação', 'sobre...",solicito gentileza informação sobre quantidade...
4,4,solicito por gentileza a informação sobre a q...,a2,1,"['solicito', 'gentileza', 'informação', 'sobre...",solicito gentileza informação sobre quantidade...


In [9]:
tokenizer = Tokenizer(num_words= max_features, split= ' ')
tokenizer.fit_on_texts(df_plain['lemma'].values)
x = tokenizer.texts_to_sequences(df_plain['lemma'].values)
x = pad_sequences(x)

In [37]:
y = pd.get_dummies(df_plain['age'])

In [38]:
X_treino, X_teste, y_treino, y_teste = train_test_split(x, y, train_size=0.8, random_state=42 )


In [39]:
model_plain = modelLSTMPos(x)

In [40]:
history = model_plain.fit(X_treino, y_treino, epochs=20, validation_data=(X_teste, y_teste))

Epoch 1/20
205/205 [==============================] - 10s 40ms/step - loss: 0.5499 - accuracy: 0.3718 - val_loss: 0.5286 - val_accuracy: 0.3799
Epoch 2/20
205/205 [==============================] - 8s 39ms/step - loss: 0.5121 - accuracy: 0.4187 - val_loss: 0.5043 - val_accuracy: 0.4299
Epoch 3/20
205/205 [==============================] - 9s 44ms/step - loss: 0.4838 - accuracy: 0.4701 - val_loss: 0.4968 - val_accuracy: 0.4433
Epoch 4/20
205/205 [==============================] - 8s 39ms/step - loss: 0.4644 - accuracy: 0.4983 - val_loss: 0.5023 - val_accuracy: 0.4421
Epoch 5/20
205/205 [==============================] - 8s 41ms/step - loss: 0.4494 - accuracy: 0.5265 - val_loss: 0.5068 - val_accuracy: 0.4439
Epoch 6/20
205/205 [==============================] - 9s 45ms/step - loss: 0.4364 - accuracy: 0.5477 - val_loss: 0.5126 - val_accuracy: 0.4323
Epoch 7/20
205/205 [==============================] - 9s 42ms/step - loss: 0.4215 - accuracy: 0.5739 - val_loss: 0.5214 - val_accuracy: 0.437

In [41]:
clf = KerasClassifier(build_fn=model_plain, epochs=5, batch_size=32, verbose=1)
resultPos = cross_val_score(clf, x, y, scoring='accuracy', cv=10)
print(resultPos)
resultPos.mean()

INFO:tensorflow:Assets written to: ram://a7e13287b0b04e69b7a416cc7bb2cfc8/assets


INFO:tensorflow:Assets written to: ram://a7e13287b0b04e69b7a416cc7bb2cfc8/assets


Epoch 1/5


2023-12-15 11:11:03.034306: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open ram://db2feb9001c14d0e8a5258e0b8f27651: INVALID_ARGUMENT: ram://db2feb9001c14d0e8a5258e0b8f27651 is a directory.
/home/chocomenta/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


231/231 [==============================] - 14s 54ms/step - loss: 0.3203 - accuracy: 0.7322
Epoch 2/5
231/231 [==============================] - 13s 56ms/step - loss: 0.2899 - accuracy: 0.7526
Epoch 3/5
231/231 [==============================] - 12s 54ms/step - loss: 0.2679 - accuracy: 0.7760
Epoch 4/5
231/231 [==============================] - 13s 54ms/step - loss: 0.2574 - accuracy: 0.7812
Epoch 5/5
26/26 [==============================] - 0s 7ms/step
INFO:tensorflow:Assets written to: ram://a27895eb6a624885836f761f2bf5d601/assets


...

INFO:tensorflow:Assets written to: ram://979c1eee24bc4a0d90e94c6365701a87/assets


Epoch 1/5


2023-12-15 11:21:37.578083: W tensorflow/core/util/tensor_slice_reader.cc:98] Could not open ram://313aa97ccb1b4df3b2269ea0b15de221: INVALID_ARGUMENT: ram://313aa97ccb1b4df3b2269ea0b15de221 is a directory.
/home/chocomenta/anaconda3/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


231/231 [==============================] - 16s 56ms/step - loss: 0.3435 - accuracy: 0.7137
Epoch 2/5
231/231 [==============================] - 13s 57ms/step - loss: 0.3083 - accuracy: 0.7423
Epoch 3/5
231/231 [==============================] - 13s 58ms/step - loss: 0.2936 - accuracy: 0.7495
Epoch 4/5
231/231 [==============================] - 13s 58ms/step - loss: 0.2775 - accuracy: 0.7663
Epoch 5/5
26/26 [==============================] - 0s 8ms/step
[0.53658537 0.57073171 0.62073171 0.52804878 0.58292683 0.58170732
 0.54390244 0.73780488 0.74634146 0.76707317]


0.6215853658536586

## BERTimbau

In [42]:
import pandas as pd
import numpy as np

In [43]:
df_plain = pd.read_csv('../data/enc_tok_nostopword_nopunct_lemm.csv')
df_plain.head()

,Unnamed: 0,req_text,age,age_encoded,word_tokens,lemma
0,0,quantos empregados em cada um dos atuais níve...,a2,1,"['quantos', 'empregados', 'cada', 'atuais', 'n...",quanto empregado cada atual nível salarial pca...
1,1,solicito cópia das atas do conselho de admini...,a2,1,"['solicito', 'cópia', 'atas', 'conselho', 'adm...",solicito cópia ata conselho administração bnde...
2,2,solicito informar a norma lei decreto portari...,a2,1,"['solicito', 'informar', 'norma', 'lei', 'decr...",solicito informar norma lei decreto portaria i...
3,3,solicito por gentileza a informação sobre a q...,a2,1,"['solicito', 'gentileza', 'informação', 'sobre...",solicito gentileza informação sobre quantidade...
4,4,solicito por gentileza a informação sobre a q...,a2,1,"['solicito', 'gentileza', 'informação', 'sobre...",solicito gentileza informação sobre quantidade...


In [44]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np



In [45]:
import  torch
from  transformers  import  AutoModel

model = AutoModel.from_pretrained('rufimelo/Legal-BERTimbau-large-TSDAE')

In [46]:
from  transformers  import  AutoTokenizer, AutoModelForMaskedLM

In [47]:
tokenizer = AutoTokenizer.from_pretrained("rufimelo/Legal-BERTimbau-large-TSDAE-v3")

In [48]:
def get_bert_embeddings(text):
    input_ids = tokenizer.encode(text, return_tensors='pt', truncation=True, padding=True, max_length=512, add_special_tokens = True)
    with  torch.no_grad():
        outs = model(input_ids)
        encoded = outs[0][0, 1:-1]
    

In [49]:
embeddings = []
for text in df_plain['lemma']:
    embeddings.append(get_bert_embeddings(text))

In [1]:
# Obtenha embeddings BERTimbau
bert_model = TFBertModel.from_pretrained("neuralmind/bert-base-portuguese-cased")
tokenized_inputs = tokenizer(df_plain['lemma'].tolist(), return_tensors="tf", padding=True, truncation=True, max_length=512)
bert_outputs = bert_model(tokenized_inputs)
embeddings = bert_outputs['last_hidden_state']

NameError: name 'TFBertModel' is not defined

In [ ]:

# Converta as classes para números usando LabelEncoder
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df_plain['age'])



In [ ]:
# Divida os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(embeddings.numpy(), labels, test_size=0.2, random_state=42)



##

In [ ]:
# Modele a LSTM
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')
])

# Compile o modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])



In [ ]:
# Treine o modelo
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

# Avalie o modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Acurácia do modelo: {accuracy}')